- Load the old tokenizer, save it locally
- Load the sentencepiece model
- Remove unwanted tokens from sentencepiece model
- Add new tokens into sentencepiece model
- Save sentencepiece model
- Load sentencepiece model into tokenizer
- Add special additional tokens into tokenizer
- Add/Remove embeddings from word embedding matrix in model
- Test

# Load the old tokenizer, save it locally

In [1]:
from transformers import AutoTokenizer

model_name = "naver-clova-ix/donut-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.save_pretrained("old_tokenizer")

print(len(tokenizer))
tokenizer

57525


XLMRobertaTokenizerFast(name_or_path='naver-clova-ix/donut-base', vocab_size=57522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s_iitcdip>', '<s_synthdog>']}, clean_up_tokenization_spaces=True)

# Load the sentencepiece model

In [2]:
from transformers.convert_slow_tokenizer import import_protobuf

model_pb2 = import_protobuf()

m = model_pb2.ModelProto()
m.ParseFromString(open("./old_tokenizer/sentencepiece.bpe.model", 'rb').read())
len(m.pieces)

57520

# Remove unwanted tokens from sentencepiece model

In [3]:
list(tokenizer.vocab.keys())[:20]

['▁1',
 '▁kilala',
 'ସୀ',
 '错误',
 '▁https',
 '▁жасаған',
 '病院',
 '▁effort',
 '▁기사',
 'ptu',
 '윽',
 'jit',
 '▁college',
 '383',
 '▁dveře',
 'masse',
 '\x9c',
 'rait',
 '闰',
 '▁gewoonlik']

In [4]:
kept_pieces = []


for p in m.pieces:

    # WRITE YOUR OWN RULE FOR WHAT TOKENS TO KEEP
    # "▁" is the prefix that means space in front of the token
    if p.piece.lstrip("▁").isascii():
        kept_pieces.append(p)

In [5]:
[kp.piece for kp in kept_pieces[:10]]

['<unk>',
 '<s>',
 '</s>',
 'a',
 '▁_',
 '▁plus',
 'II',
 '▁own',
 'enseignement',
 'ective']

In [6]:
i = 0

kept_tokens = set([x.piece for x in kept_pieces])

while i < len(m.pieces):

    if m.pieces[i].piece not in kept_tokens:
        m.pieces.pop(i)
    else:
        i += 1


In [7]:
len(m.pieces)

27510

# Add new tokens into sentencepiece model

#### The Donut tokenizer doesn't have the "1" token

It has tokens for " 1", "10", and "1.1", but certain scenarios result in the UNK token being used

In [8]:
print(tokenizer.unk_token_id)

# This results in the token turning into an unknown token (3)
tokenizer(">1").input_ids

3


[0, 56881, 3, 2]

In [9]:
# Whenever a character is before the number 1, there is a decent chance the 1 will turn into UNK (id = 3)
ids = tokenizer("10.1 )1 a1 10 1 11").input_ids
tokens = tokenizer.tokenize("10.1 )1 a1 10 1 11", add_special_tokens=True)

list(zip(ids, tokens))

[(0, '<s>'),
 (39772, '▁10.'),
 (3, '1'),
 (9447, '▁)'),
 (3, '1'),
 (54915, '▁a'),
 (3, '1'),
 (23485, '▁10'),
 (1314, '▁1'),
 (44028, '▁11'),
 (2, '</s>')]

In [10]:
[x for x in m.pieces if x.piece == "▁1"]

[piece: "\342\226\2011"
 score: -7.551031589508057
 type: NORMAL]

In [11]:
from copy import deepcopy

# copy the last piece
piece1 = deepcopy(m.pieces[-1])

# modify the values of the following variables
piece1.piece = "1"
piece1.score = -10

# include it in the models list of pieces
m.pieces.extend([piece1])

In [12]:
len(m.pieces)

27511

# Save sentencepiece model

In [13]:
with open("temp_sentencepiece.bpe.model", 'wb') as f:
    f.write(m.SerializeToString())

# Load sentencepiece model into tokenizer

In [14]:
from transformers import XLMRobertaTokenizer

new_tokenizer = XLMRobertaTokenizer(vocab_file="temp_sentencepiece.bpe.model")

In [15]:
len(new_tokenizer), len(tokenizer)

(27513, 57525)

# Add special additional tokens into tokenizer

In [16]:
tokenizer.additional_special_tokens

['<s_iitcdip>', '<s_synthdog>']

In [17]:
new_tokenizer.add_special_tokens({'additional_special_tokens': tokenizer.additional_special_tokens})

2

In [18]:
new_tokenizer.tokenize("<s>")

['▁<', 's', '>']

In [19]:
new_tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': ['<s_iitcdip>', '<s_synthdog>']}

In [20]:
# the special tokens are in the model, but due to a quirk, they need to be added again

new_tokenizer.add_special_tokens(new_tokenizer.special_tokens_map)

new_tokenizer.save_pretrained('donut-base-ascii')

('donut-base-ascii/tokenizer_config.json',
 'donut-base-ascii/special_tokens_map.json',
 'donut-base-ascii/sentencepiece.bpe.model',
 'donut-base-ascii/added_tokens.json')

In [21]:
new_tokenizer.tokenize("<s>")

['<s>']

In [22]:
len(new_tokenizer), len(tokenizer)

(27515, 57525)

In [23]:
new_tokenizer = AutoTokenizer.from_pretrained("donut-base-ascii")
new_tokenizer.tokenize("<s>")

['<s>']

# Add/Remove embeddings from word embedding matrix in model

### Step 1: Create mapping of old ids to new ids

This is necessary because word embeddings are being deleted.

e.g. what used to be 54000 is now 23000

In [24]:
old_mapping = tokenizer.vocab

new_mapping = new_tokenizer.vocab

sorted_new_mapping = sorted(new_mapping.items(), key=lambda x: x[1])# sort by id, ascending

# `embed_indexes` will have the old index value stored at the new index
# e.g. embed_indexes[i] = j means the new embedding id at i has the same value
# as the old embedding id of j
embed_indexes = [old_mapping[tok] for tok, _ in sorted_new_mapping[:-4]]

In [25]:
# embed_indexes ignores the last 4 because
# the embedding for "1" is brand new.

# the other embeddings will get added later
sorted_new_mapping[-4:]

[('1', 27511),
 ('<mask>', 27512),
 ('<s_iitcdip>', 27513),
 ('<s_synthdog>', 27514)]

### Step 2: Copy embeddings to keep

In [26]:
from transformers import VisionEncoderDecoderModel

model_name = "naver-clova-ix/donut-base"
model = VisionEncoderDecoderModel.from_pretrained(model_name)

old_embeds =  model.decoder.model.decoder.embed_tokens.weight.data
old_embeds

new_embeds = old_embeds[embed_indexes, :].clone()

print(new_embeds.shape)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/nicholas/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/share/gconf/ubuntu.default.path')}
  warn(msg)
/home/nicholas/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your p

torch.Size([27511, 1024])


In [27]:
embed_indexes[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]

### Step 3: Add new embeddings (if necessary)


The new token doesn't have an old embedding, so we will copy a similar embedding.

In [28]:
import torch

# setting the embedding for the new token to be the same as " 1"
# during training, they will differentiate
embed_1 = old_embeds[old_mapping["▁1"]].clone().unsqueeze(0)
print(embed_1.shape)

additional_embeds = []
for tok, _ in sorted_new_mapping[-3:]:
    additional_embeds.append(old_embeds[old_mapping[tok]].clone().unsqueeze(0))
    

new_embeds = torch.vstack([new_embeds, embed_1, *additional_embeds])

new_embeds.shape

torch.Size([1, 1024])


torch.Size([27515, 1024])

### Step 4: Making the embeddings a multiple of 16

[It can provide a minor speedup when training or doing inference.](https://twitter.com/karpathy/status/1621578354024677377?s=20)

In [29]:
while new_embeds.shape[0] % 16 != 0:
    new_embeds = torch.vstack([new_embeds, torch.zeros(1, new_embeds.shape[1])])


new_embeds.shape

torch.Size([27520, 1024])

In [30]:
27520/64

430.0

### Step 5: Put embeddings back into model

In [31]:
model.decoder.model.decoder.embed_tokens.weight.data = new_embeds

model.decoder.config.update({
    "vocab_size": new_embeds.shape[0]
})

model.save_pretrained("donut-base-ascii")

### Step 6: Making sure the embeddings are correct

In [32]:
old_ids = tokenizer("hello there").input_ids
print(old_ids)

new_ids = new_tokenizer("hello there").input_ids
print(new_ids)

[0, 37199, 35816, 34554, 2]
[0, 14026, 13045, 12147, 2]


In [33]:
import torch

old_embeddings = torch.stack([old_embeds[i] for i in old_ids])
new_embeddings = torch.stack([new_embeds[i] for i in new_ids])

torch.all(torch.eq(old_embeddings, new_embeddings))

tensor(True)

### Add image processor so that all files are together

In [34]:
from transformers import AutoImageProcessor

proc = AutoImageProcessor.from_pretrained(model_name)
proc.save_pretrained("donut-base-ascii")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


['donut-base-ascii/preprocessor_config.json']

### Check that the new token for 1 works


unk_token_id = 3, so that shouldn't be present! Instead it should have 27511, the new token for "1"

In [35]:
ids = new_tokenizer("10.1 )1 a1 10 1 11").input_ids
tokens = new_tokenizer.tokenize("10.1 )1 a1 10 1 11", add_special_tokens=True)

list(zip(ids, tokens))

[(0, '<s>'),
 (15793, '▁10.'),
 (27511, '1'),
 (4056, '▁)'),
 (27511, '1'),
 (26020, '▁a'),
 (27511, '1'),
 (9515, '▁10'),
 (557, '▁1'),
 (18739, '▁11'),
 (2, '</s>')]

# Test

In [36]:
from transformers import AutoProcessor, VisionEncoderDecoderModel

model_name = "naver-clova-ix/donut-base"

processor = AutoProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device);

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [37]:
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/example-documents", split="test")

image = dataset[1]["image"]

task_prompt = "<s_synthdog>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

In [38]:
image

In [39]:
%%timeit -n 5 -r 2

pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,   
    min_length=10,
    max_length=10
)

467 ms ± 42.1 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [40]:
pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,   
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence

/home/nicholas/miniconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<s_synthdog> LOVE DELIGHTFULLY SOFT SKIN? GET INDIAS FIR</s>'

In [41]:
model_name = "./donut-base-ascii"

new_processor = AutoProcessor.from_pretrained(model_name)
new_model = VisionEncoderDecoderModel.from_pretrained(model_name)

device = 0

new_model.to(device);

new_decoder_input_ids = new_processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
new_decoder_input_ids

tensor([[27514]])

In [42]:
%%timeit -n 5 -r 2

pixel_values = new_processor(image, return_tensors="pt").pixel_values

outputs = new_model.generate(
    pixel_values.to(device),
    decoder_input_ids=new_decoder_input_ids.to(device),
    early_stopping=True,
    pad_token_id=new_processor.tokenizer.pad_token_id,
    eos_token_id=new_processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[new_processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,   
    min_length=10,
    max_length=10
)

382 ms ± 2.19 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [43]:
pixel_values = new_processor(image, return_tensors="pt").pixel_values

outputs = new_model.generate(
    pixel_values.to(device),
    decoder_input_ids=new_decoder_input_ids.to(device),
    early_stopping=True,
    pad_token_id=new_processor.tokenizer.pad_token_id,
    eos_token_id=new_processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=1,
    bad_words_ids=[[new_processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,   
)

sequence = new_processor.batch_decode(outputs.sequences)[0]
sequence

'<s_synthdog> LOVE DELIGHTFULLY SOFT SKIN? GET INDIAS FIR</s>'